In [10]:
import pandas as pd
import numpy as np
import datasets
from inference_utils import *
import json
import os
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)
import utils
from utils import  get_stats

### read jsonl file
aspect = 'all'

data = datasets.load_dataset('boda/review_evaluation_automatic_labels', aspect)['test']

gold_data = get_gold_labels(data, aspect,'chatgpt_ASPECT_score')





{'actionability_label': '3', 'grounding_specificity_label': '3', 'verifiability_label': '3', 'helpfulness_label': '3'}


README.md:   0%|          | 0.00/3.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/19.8M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [9]:
import json
import re

def process_jsonl(file_path):
    aspect_scores = []
    errors = 0
    total = 0
    
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data = json.loads(line)
            text = data.get("generated_text", "")
            
            scores = {}
            for aspect in ["Actionability", "Grounding Specificity", "Verifiability", "Helpfulness"]:
                total += 1
                aspect_lower = aspect.lower().replace(" ", "_")
                match = re.search(rf"(?:\*\*{aspect}:\s*(\d+)\*\*|\*\*{aspect}\*\*:\s*(\d+)|{aspect}:\s*(\d+))", text, re.IGNORECASE)
                if match:
                    scores[aspect_lower] = int(next(g for g in match.groups() if g is not None))
                else:
                    scores[aspect_lower] = None  # Handle missing scores
                    errors += 1
            
            aspect_scores.append(scores)
    
    print(f"Processed {len(aspect_scores)} scores with {errors} errors out of {total} total scores")
    return aspect_scores

# Example usage
scores = process_jsonl("/home/abdelrahman.sadallah/review_rewrite/inference/evalute_outputs/adapters/DeepSeek-R1-Distill-Qwen-7B/score_only/chat/all/raw_outputs.jsonl")
for score in scores:

    print(score)

Processed 1000 scores with 2075 errors out of 4000 total scores
{'actionability': 3, 'grounding_specificity': None, 'verifiability': 1, 'helpfulness': 3}
{'actionability': 1, 'grounding_specificity': 5, 'verifiability': 1, 'helpfulness': 2}
{'actionability': None, 'grounding_specificity': None, 'verifiability': None, 'helpfulness': None}
{'actionability': 3, 'grounding_specificity': 2, 'verifiability': 1, 'helpfulness': 3}
{'actionability': None, 'grounding_specificity': None, 'verifiability': None, 'helpfulness': None}
{'actionability': None, 'grounding_specificity': None, 'verifiability': None, 'helpfulness': None}
{'actionability': None, 'grounding_specificity': None, 'verifiability': None, 'helpfulness': None}
{'actionability': 5, 'grounding_specificity': None, 'verifiability': 1, 'helpfulness': 4}
{'actionability': 3, 'grounding_specificity': 2, 'verifiability': 1, 'helpfulness': 3}
{'actionability': 5, 'grounding_specificity': None, 'verifiability': 5, 'helpfulness': 5}
{'actiona

In [2]:

model_outputs = []
parent_dir = 'evalute_outputs/full_model/scilitllm/score_only/instruction/all/label_loss/'
data_path =   parent_dir + 'predictions.jsonl'


with open(data_path) as f:
    for line in f:
        model_outputs.append(json.loads(line))
        

In [3]:
assert len(model_outputs) == len(gold_data)

In [4]:
aspect

'all'

In [15]:
aspect = 'all'
aspects = [ 'actionability', 'grounding_specificity','verifiability', 'helpfulness'] if aspect == 'all' else [aspect]
## convert the list of dictionaries to dictionary of lists

model_outputs =process_jsonl("/home/abdelrahman.sadallah/review_rewrite/inference/evalute_outputs/adapters/DeepSeek-R1-Distill-Qwen-7B/score_only/chat/all/raw_outputs.jsonl")




predctions = {aspect:[] for aspect in aspects}
labels = {aspect:[] for aspect in aspects}

for i in range(len(model_outputs)):
    for aspect in aspects:

        l = gold_data[i][aspect]
        p = model_outputs[i][f'{aspect}']
        l = str(l)
        p = str(p)

        if p == 'None':
            continue
        predctions[aspect].append(p)
        labels[aspect].append(l)

for aspect in aspects:
    print(len(predctions[aspect]), len(labels[aspect]))

parent_dir = "/home/abdelrahman.sadallah/review_rewrite/inference/evalute_outputs/adapters/DeepSeek-R1-Distill-Qwen-7B/score_only/chat/all/"

Processed 1000 scores with 2075 errors out of 4000 total scores
563 563
263 263
535 535
564 564


In [16]:
## write the stats into file
with open(f'{parent_dir}evalute_outputs.txt', 'w') as f:

    f.write('number of samples: ' + str(len(labels[aspect])) + '\n')
    for a in aspects:
        stat_dict = get_stats(predctions[a], labels[a], a)
        f.write('*' * 20 + f'{a}'+ '*' * 20 + '\n')

        ## remove the acc from the dict, and for spearman, only include the correlation

        for k,v in stat_dict.items():
            if 'accuracy' in k:
                continue
            if 'spearman' in k:
                v = v[0]
            ## round the values
            if isinstance(v, float):
                v = round(v, 3)
            f.write(f'{k}: {v}\n')
        f.write('-'*50+'\n')



In [7]:
# data = datasets.load_dataset('boda/review_evaluation_automatic_labels', 'all')




In [8]:
# import pandas as pd
# import matplotlib.pyplot as plt

# # Convert score values to numeric before computing value counts
# value_counts = pd.DataFrame({
#     aspect: pd.to_numeric(data['train'].to_pandas()[f'chatgpt_{aspect}_score'], errors='coerce').value_counts()
#     for aspect in aspects
# })

# # Sort index to ensure correct order
# value_counts = value_counts.sort_index()

# # Plot the combined value counts
# value_counts.plot(kind='bar', title='Aspect Distribution')
# plt.xlabel('Scores')
# plt.ylabel('Counts')
# plt.show()


In [9]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("allenai/scitulu-7b")

def print_tokens_with_ids(txt):
    tokens = tokenizer.tokenize(txt, add_special_tokens=False)
    token_ids = tokenizer.encode(txt, add_special_tokens=False)
    print(list(zip(tokens, token_ids))[0:])

prompt = """###Review Point:
3. The theoretical results are based on certain assumptions that may not hold in all practical scenarios. 
###Output:
{'actionability_label':"""
print_tokens_with_ids(prompt)  # [..., ('▁Hello', 15043), ('<0x0A>', 13), ('<0x0A>', 13), ('##', 2277), ('#', 29937), ('▁Ass', 4007), ('istant', 22137), (':', 29901), ...]

response_template = "\n\n###Output:\n"
print_tokens_with_ids(response_template)  # [('▁###', 835), ('▁Ass', 4007), ('istant', 22137), (':', 29901)]

tokenizer(response_template)['input_ids'][2:]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggin

[('▁###', 835), ('Re', 1123), ('view', 1493), ('▁Point', 8984), (':', 29901), ('<0x0A>', 13), ('3', 29941), ('.', 29889), ('▁The', 450), ('▁theoretical', 23399), ('▁results', 2582), ('▁are', 526), ('▁based', 2729), ('▁on', 373), ('▁certain', 3058), ('▁assumptions', 20813), ('▁that', 393), ('▁may', 1122), ('▁not', 451), ('▁hold', 4808), ('▁in', 297), ('▁all', 599), ('▁practical', 15031), ('▁scenarios', 21846), ('.', 29889), ('▁', 29871), ('<0x0A>', 13), ('##', 2277), ('#', 29937), ('Output', 6466), (':', 29901), ('<0x0A>', 13), ("{'", 10998), ('action', 2467), ('ability', 3097), ('_', 29918), ('label', 1643), ("':", 2396)]
[('▁', 29871), ('<0x0A>', 13), ('<0x0A>', 13), ('##', 2277), ('#', 29937), ('Output', 6466), (':', 29901), ('<0x0A>', 13)]


[13, 13, 2277, 29937, 6466, 29901, 13]

In [2]:
import datasets

ds = datasets.load_dataset('boda/review_evaluation_human_annotation', name='helpfulness', split='gold')

gold-00000-of-00001.parquet:   0%|          | 0.00/302k [00:00<?, ?B/s]

silver-00000-of-00001.parquet:   0%|          | 0.00/714k [00:00<?, ?B/s]

hard-00000-of-00001.parquet:   0%|          | 0.00/315k [00:00<?, ?B/s]

Generating gold split:   0%|          | 0/211 [00:00<?, ? examples/s]

Generating silver split:   0%|          | 0/589 [00:00<?, ? examples/s]

Generating hard split:   0%|          | 0/254 [00:00<?, ? examples/s]

In [11]:
import re
import json

def escape_inner_quotes(text):
    """Finds specified rationale fields and escapes only internal double quotes."""
    fields = [
        "actionability_rationale",
        "grounding_specificity_rationale",
        "verifiability_rationale",
        "helpfulness_rationale"
    ]
    
    for field in fields:
        pattern = fr'("{field}"\s*:\s*")(.*?)("[\}},])'  # Escape closing brace
        matches = list(re.finditer(pattern, text, re.DOTALL))  # Find all matches first
        
        for match in reversed(matches):  # Process from last to first to avoid index shifting
            before, rationale, after = match.groups()
            escaped_rationale = rationale.replace('"', '\\"')  # Escape only inner quotes
            text = text[:match.start(2)] + escaped_rationale + text[match.end(2):]
    
    return text

def extract_dict(text):


    ## remove double spaces
    text = re.sub(' +', ' ', text)
    ## remove ``` from the text
    text = text.replace('```', '')
    text = text.replace("-", "")  # Remove leading hyphens
    text = text.replace("\n", " ")  # Remove newlines
    text = text.replace("\\'", "'")  # Fix incorrectly escaped single quotes
    text = text.replace('\\"s', "'s")  # Fix incorrect escaped possessive 's
    text = text.replace("\\\\'", "\\\"")
    text = text.replace("'", '"')  # Replace single quotes with double quotes

    ## if text begin with comma or space, remove it
    if text[0] == ',' or text[0] == ' ':
        text = text[1:]
    ## if the text doesn't begin with {, then add it
    if text[0] != '{':
        text = '{' + text + '}'

    text = escape_inner_quotes(text)  # Fix quotes inside rationale fields
    # print(text)

    match = re.search(r'\{.*?\}', text, re.DOTALL)  # Extract first {...} block
    if match:
        dict_str = match.group()  # Get extracted dictionary string
        
        try:
            return json.loads(dict_str)  # Convert to Python dictionary safely
        except json.JSONDecodeError as e:
            print(f"Parsing error: {e}\nProblematic string: {dict_str}")
            return None
    
    return None  # Return None if no dictionary found




def extract_predictions(model_outputs):
    """
    Parses a list of model-generated texts to extract labels and returns a dictionary.
    
    :param model_outputs: List of strings containing model-generated text with labels.
    :return: List of dictionaries with extracted labels.
    """
    errors= 0
    extracted_data = []
    
    for text in model_outputs:

        if type(text) == dict:
            if 'outputs' in text.keys():
                text = text.outputs[0].text
            elif 'generated_text' in text.keys():
                text = text['generated_text']

        extracted_dict = extract_dict(text)
        if not extracted_dict:
            errors += 1
    print(f"Extracted {len(extracted_data)} labels with {errors} errors")

outputs = []
with open('/home/abdelrahman.sadallah/mbzuai/review_rewrite/inference/evalute_outputs/base_model/Llama-3.1-8B/score_only/instruction/step-0/review_evaluation_automatic_labels/raw_outputs_all_test.jsonl', 'r') as f:
    for line in f:
        outputs.append(json.loads(line))

# outputs = ["""   \"actionability_label\": \"2\",\n   \"grounding_specificity_label\": \"4\",\n   \"verifiability_label\": \"4\",\n   \"helpfulness_label\": \"4\"\n"""]

extract_predictions(outputs)


Parsing error: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
Problematic string: {actionability_label: 3  grounding_specificity_label: 3  verifiability_label: 3  helpfulness_label: 3 }
Parsing error: Expecting ',' delimiter: line 1 column 123 (char 122)
Problematic string: {"actionability_label": "4",  "grounding_specificity_label": "4",  "verifiability_label": "4",  "helpfulness_label": "4"  ###Rationale: The review point is mostly actionable as it suggests conducting more experiments on diverse datasets to demonstrate the generalization capabilities of the model. The review point is fully grounded and specific as it explicitly mentions the need for more experiments on diverse datasets to demonstrate the generalization capabilities of the model. The review point is fully verifiable as it provides a clear suggestion for conducting more experiments on diverse datasets to demonstrate the generalization capabilities of the model. The review point is highly he

In [13]:
import os
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)
import utils
from utils import get_prompt

row = {'review_point': 'I think the author "should" provide more examples to "support" their argument.', 'chatgpt_actionability_score': 1.0, 'chatgpt_actionability_rationale': 'The author should provide more examples "to" support their argument.', 'chatgpt_grounding_specificity_score': 1.0, 'chatgpt_grounding_specificity_rationale': 'The author should provide more examples to support their argument.', 'chatgpt_verifiability_score': 1.0, 'chatgpt_verifiability_rationale': 'The author should provide more examples to support their argument.', 'chatgpt_helpfulness_score': 1.0, 'chatgpt_helpfulness_rationale': 'The author should provide more examples to support their argument.'}
pr = get_prompt(row,aspect= 'all',task='train', generation_type='score_rationale', prompt_type = 'instruction')

with open('prompt.txt', 'w') as f:
    if isinstance(pr, list):
        for item in pr:
            f.write("%s\n" % item)
    else:
        f.write(pr['text'])

TypeError: 'set' object is not subscriptable